In [28]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import re
import string
#import emoji
#from PIL import Image
from collections import Counter
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier,RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC,LinearSVC
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

In [2]:
df = pd.read_csv("C:\\Datasets\\cyberbullying_tweets\\cyberbullying_tweets.csv")

In [3]:
!pip install emoji

ERROR: Could not install packages due to an OSError: HTTPSConnectionPool(host='files.pythonhosted.org', port=443): Max retries exceeded with url: /packages/05/73/0b381e376dbd396717bd2963eda0f674ff8d1af840faa83e7020c2fc55f9/emoji-2.5.1.tar.gz (Caused by NewConnectionError('<pip._vendor.urllib3.connection.HTTPSConnection object at 0x00000228A13571C0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))



In [3]:
df.head()

,tweet_text,cyberbullying_type
0,"In other words #katandandre, your food was cra...",not_cyberbullying
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying


In [4]:
df.isnull().sum()

tweet_text            0
cyberbullying_type    0
dtype: int64

In [5]:
df.dtypes

tweet_text            object
cyberbullying_type    object
dtype: object

In [6]:
df['cyberbullying_type'].value_counts().head()

religion             7998
age                  7992
gender               7973
ethnicity            7961
not_cyberbullying    7945
Name: cyberbullying_type, dtype: int64

In [7]:
df['tweet_length'] = df['tweet_text'].apply(len)

In [8]:
df.head()

,tweet_text,cyberbullying_type,tweet_length
0,"In other words #katandandre, your food was cra...",not_cyberbullying,61
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying,115
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying,60
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying,103
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying,103


In [9]:
df

,tweet_text,cyberbullying_type,tweet_length
0,"In other words #katandandre, your food was cra...",not_cyberbullying,61
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying,115
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying,60
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying,103
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying,103
...,...,...,...
47687,"Black ppl aren't expected to do anything, depe...",ethnicity,234
47688,Turner did not withhold his disappointment. Tu...,ethnicity,276
47689,I swear to God. This dumb nigger bitch. I have...,ethnicity,104
47690,Yea fuck you RT @therealexel: IF YOURE A NIGGE...,ethnicity,90


In [10]:
df['tweet_length'].describe()

count    47692.000000
mean       136.253229
std         85.226899
min          1.000000
25%         78.000000
50%        124.000000
75%        180.000000
max       5018.000000
Name: tweet_length, dtype: float64

In [11]:
# Renaming our columns
df = df.rename(columns={'tweet_text':'text','cyberbullying_type':'sentiment','tweet_length':'length'})

In [13]:
df.head()

,text,sentiment,length
0,"In other words #katandandre, your food was cra...",not_cyberbullying,61
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying,115
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying,60
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying,103
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying,103


In [30]:
df['sentiment'].value_counts()

religion               7998
age                    7992
gender                 7973
ethnicity              7961
not_cyberbullying      7945
other_cyberbullying    7823
Name: sentiment, dtype: int64

In [32]:
# First we can encode our sentiments
df['encoded'] = df['sentiment'].replace(to_replace={'religion':1,'age':2,'gender':3,'ethinicity':4,'not_cyberbullying':5,'other_cyberbullying':6})

In [33]:
df['encoded'].head()

0    5
1    5
2    5
3    5
4    5
Name: encoded, dtype: object

In [34]:
df.head(10)

,text,sentiment,length,encoded
0,"In other words #katandandre, your food was cra...",not_cyberbullying,61,5
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying,115,5
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying,60,5
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying,103,5
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying,103,5
5,"@Raja5aab @Quickieleaks Yes, the test of god i...",not_cyberbullying,131,5
6,Itu sekolah ya bukan tempat bully! Ga jauh kay...,not_cyberbullying,54,5
7,Karma. I hope it bites Kat on the butt. She is...,not_cyberbullying,63,5
8,@stockputout everything but mostly my priest,not_cyberbullying,44,5
9,Rebecca Black Drops Out of School Due to Bully...,not_cyberbullying,50,5


# TEXT PREPROCESSING
Text preprocessing is going be a five step process
1.Removing al URLs,non utf characters
2.Decontracting the text
3.Cleaning all the hashtag
4.Filtering all the special characters such as $ and &
5.Removing Multiple sequence spaces
6.Stemming of Words
7.Lemmatization of Words
8.Removing all the emojis 

In [69]:
# Removing all  the emojis 
#def strip_all_emoji(text):
    #return emoji.replace_emoji(text,replace="")
# Removing all the Urls ,symbols and non-utf8 characters
def strip_all_entities(text):
    text = text.replace('\r', '').replace('\n', ' ').lower()
    text = re.sub(r"(?:\@|https?\://)\S+", "", text)
    text = re.sub(r'[^\x00-\x7f]',r'', text)
    text = re.sub(r'(.)1+', r'1', text)
    text = re.sub('[0-9]+', '', text)
    stopchar =string.punctuation
    table = str.maketrans('','',text)
    text = text.translate(table)
    text =[word for word in text.split() if word not in stop_words]
    text = ''.join(text)
# Decontracting the text for words such as cant,am ,i will ,making shortforms longer so that the model understands more
def decontract(text):
    text = re.sub(r"can\'t","can not",text)
    text = re.sub(r"n\'t"," not ",text)
    text  = re.sub(r"\'re"," are ",text)
    text = re.sub(r"\'d"," would ",text)
    text = re.sub(r"\'ll"," will ",text)
    text = re.sub(r"\'ve"," have ",text)
    text = re.sub(r"\'m", " am ",text)
    text = re.sub(r"\'t"," not ",text)
    return text
# Cleaning all the hashtags
def clean_hashtags(text):
    new_tweet = ''.join(word.strip() for word in re.split('#(?!(?:hashtag)\b)[\w-]+(?=(?:\s+#[\w-]+)*\s*$)',tweet))
    new_tweet_2 = "".join(word.strip() for word in re.split('# |_',new_tweet))
    return new_tweet_2
# A function that removes special chars such as $ and #
def remove_chars (a):
    sent =  []
    for word in a.split(''):
        if (('$') in word |('#') in word ):
            sent.append(a)
        else :
            sent.append(word)
        return ''.join(word)
# Removing multiple sequence spaces
def remove_spaces(text):
    return re.sub(r"\s\s+","",text)
# Applying stemming to words 
def stemming(text) :
    tokenized = word_tokenize(text)
    ps = PorterStemmer()
    return ''.join([ps.stem(words)for words in tokenizer])
# Applying Lemmatization
def lemmatization(text):
    tokenized = word_tokenizer(text)
    lm = WordNetLemmatizer()
    return ''.join([lm.lemmatize(words)for words in tokenized])



In [70]:
# A function that does all the preprocessing built on all  the other function
def TextPreprocess(text):
    #text = strip_all_emoji(text)
    text = strip_all_entities(text)
    text = decontract(text)
    text = clean_hashtags(text)
    text = remove_chars(text)
    text = remove_spaces(text)
    text = stemming(text)
    text = lemmatization(text)

In [71]:
df.copy()
sample_df= df.head(20)

In [72]:
sample_df.head()

,text,sentiment,length,encoded
0,"In other words #katandandre, your food was cra...",not_cyberbullying,61,5
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying,115,5
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying,60,5
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying,103,5
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying,103,5


In [73]:
#sample_df['cleaned_text'] = sample_df['text'].apply(TextPreprocess)

TypeError: expected string or bytes-like object

In [1]:
# Splitting Data into CyberBullying and Not CyberBullying
not_cyberbullying =df[df['sentiment']=='not_cyberbullying']
age_type = df[df['sentiment']=='age']
gender_type = df[df['sentiment']=='gender']
ethnicity_type = df[df['sentiment']=='ethinicity']
other_cyberbullying = df[df['sentiment']=='other_cyberbullying']
religion_type = df[df['sentiment']=='religion']


NameError: name 'df' is not defined

In [ ]:
df['cleaned_text'] = df['text'].apply(preprocess)
df.head()

In [ ]:
df.drop_duplicates("cleaned_text", inplace=True)

In [ ]:
text_len = []
for text in df.tweet_list:
    tweet_len = len(text)
    text_len.append(tweet_len)
data['text_len'] = text_len

In [ ]:
text_len = []
for text in df.tweet_list:
    tweet_len = len(text)
    text_len.append(tweet_len)
df['text_len'] = text_len

# EXPLORATORY DATA ANALYSIS

In [ ]:
from collections import Counter
gender = Counter([item for sublist in gender_type['tweet_list']for item in sublist])
top20_gender = pd.DataFrame(gender.most_common(20))
top20_gender.columns = ['TopWords','Frequency']
fig = plt.figure(figsize=(15,8))
sns.barplot(data=top20_gender,x='TopWords',y='Frequency')
plt.title('Top 20 Words in Gender CyberBullying')
plt.show()

In [ ]:
age = Counter([item for sublist in age_type['tweet_list']for item in sublist])
top20_age = pd.DataFrame(age.most_common(20))
top20_age.columns = ['TopWords','Frequency']
fig = plt.figure(figsize=(15,8))
sns.barplot(data=top20_age,x='TopWords',y='Frequency')
plt.title('The Most appearnig words in Age Discrimination')
plt.show()

In [ ]:
religion = Counter([item for sublist in religion_type['tweet_list']for item in sublist])
top20_religion = pd.DataFrame(religion.most_common(20))
top20_religion.columns = ['TopWords','Frequency']
fig = plt.figure(figsize=(15,8))
sns.barplot(data=top20_religion,x='TopWords',y='Frequency')
plt.title('Top 20 Text in Religion Cyberbullying')
plt.show()

In [ ]:
ethnicity = Counter([item for sublist in ethnicity['tweet_list']for item in sublist])
top20_ethnicity = pd.DataFrame(ethnicity.most_common(20))
top20_ethnicity.columns = ['TopWords','Frequency']
fig = plt.figure(figsize=(15,8))
sns.barplot(data=top20_ethnicity,x='TopWords',y='Frequency')
plt.title('Top 20 Text that signify Ethinicity Profiling')
plt.show()

In [ ]:
not_cyberbullying = Counter([item for sublist in not_cyberbullying['tweet_list']for item in sublist])
top20_notcyberbullying = pd.DataFrame(not_cyberbulling.most_common(20))
top20_notcyberbullying.columns = ['TopWords'.'Frequency']
fig = plt.figure(figsize=(15,8))
sns.barplot(data=top20_notcyberbullying,x='TopWords',y='Frequency')
plt.title('Texts that were classified as not cyberbullying')

# MODEL TRAINING :: ENSEMBLE MODELS,TREE ,NAIVE BAYES ,SVM

In [ ]:
# Splitting Data into Training and Testing Set
X, Y = df['cleaned_text'],df['encoded']
X_train,X_test,y_train,y_test = train_test_split(X,y.stratify=Y,test_size=0.3,random_state=104)


In [ ]:
# TFIDF VECTORIZATION FOR OUR CLEANED TEXT
tfidf = TfidfVectorizer()
X_train_tf =  tfidf.fit_transform(X_train)
X_test_tf = tfidf.fit_transform(X_test)


# MACHINE LEARNING MODELS

In [ ]:
lr = LogisticRegression()
lr_cv_score = cross_val_score(lr,X_train,y_train,cv=5,score='f1_macro',n_jobs=1)
mean_lr_cv_score = np.mean(lr_cv_score)

In [ ]:
lin_svc = LinearSVC()
lin_svc_score = cross_val_score(lin_svc,X_train,y_train,cv=5,score='f1_macro',n_jobs=1)
mean_lin_svc_score = np.mean(lin_svc_score)


In [ ]:
multiNB =MultiNomialNB()
multiNB_score = cross_val_score(multiNB,X_train,y_train,cv=5,score='f1_macro',n_jobs=1)
mean_multiNB = np.mean(multiNB_score)

In [ ]:
rf = RandomForestClassifier()
rf_score = cross_val_score(rf,X_train,y_train,score='f1_macro',n_jobs=1)
mean_rf_score = np.mean(rf_score)

In [ ]:
dt = DecisionTreeClassifier()
dt_score = cross_val_score(dt,X_train,y_train,score='f1_macro',n_jobs=1)

In [ ]:
#print("Classification Report:/n",classification_report(y))